<a href="https://colab.research.google.com/github/aditidagar/ocular_disease_recog/blob/main/PREPROCESSING/FINAL_FUSION_WORK_OCULAR_DISEASE_AUGMENTATION_CLAHE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

NOTE: This script is forked from Grezgor Meller's work: https://github.com/GrzegorzMeller/AlgorithmsForMassiveData/blob/master/Preprocessing/OCULAR_DISEASE_AUGMENTATION.ipynb

Modifications were made to account for image fusion methods, though we had planned to improve the augmentation process further by using Albumentations. 

In [ ]:
from google.colab import drive
drive.mount('/amd/')

Mounted at /amd/


NOTE: We assume you have a directory called "ML490" in your Google Drive which contains the pre-processed data. If you do not, adjust the path below to point to the pre-processed data

In [ ]:
!cp /amd/My\ Drive/ML490/ODIR-5K_contrast.zip /content/

In [ ]:
!unzip ODIR-5K_contrast.zip

In [ ]:
import glob
def show_class_distribution():
  N = 0
  D=0
  G=0
  C=0
  A=0
  H=0
  M=0
  O = 0
  X = 0
  for element in glob.glob("ODIR-5K/Training Images/*.jpg"):
    img_name = element.split("/")[-1]
    img_label = img_name.split("-")
    if img_label[-1] == "N.jpg":
      N += 1
    elif img_label[-1] == "D.jpg":
      D +=1
    elif img_label[-1] == "G.jpg":
      G +=1
    elif img_label[-1] == "C.jpg":
      C +=1
    elif img_label[-1] == "A.jpg":
      A +=1
    elif img_label[-1] == "H.jpg":
      H +=1
    elif img_label[-1] == "M.jpg":
      M +=1
    elif img_label[-1] == "O.jpg":
      O +=1
    elif "&" in img_label[-1]:
      X +=1  
  print(N, D, G, C, A, H, M, O, X)
show_class_distribution()

1999 991 149 173 159 62 166 536 379


In [ ]:
# Only one of these lines should be uncommented, depending on which image fusion method you are using. It must be consistent with the fusionMethod variable used in the pre-processing script
# fusionMethod = "CONCAT"
# fusionMethod = "SUM"
 fusionMethod = "NONE"

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import random
from PIL import Image, ImageEnhance


if fusionMethod == "CONCAT":
  # We can't rotate images when using concatenation fusion method since the images are rectangular
  MAX_ROTATE = 0
else:
  MAX_ROTATE = 10

def cv2_clipped_zoom(img, zoom_factor):
    height, width = img.shape[:2] # It's also the final desired shape
    new_height, new_width = int(height * zoom_factor), int(width * zoom_factor)
    y1, x1 = max(0, new_height - height) // 2, max(0, new_width - width) // 2
    y2, x2 = y1 + height, x1 + width
    bbox = np.array([y1,x1,y2,x2])
    # Map back to original image coordinates
    bbox = (bbox / zoom_factor).astype(np.int)
    y1, x1, y2, x2 = bbox
    cropped_img = img[y1:y2, x1:x2]

    # Handle padding when downscaling
    resize_height, resize_width = min(new_height, height), min(new_width, width)
    pad_height1, pad_width1 = (height - resize_height) // 2, (width - resize_width) //2
    pad_height2, pad_width2 = (height - resize_height) - pad_height1, (width - resize_width) - pad_width1
    pad_spec = [(pad_height1, pad_height2), (pad_width1, pad_width2)] + [(0,0)] * (img.ndim - 2)

    result = cv.resize(cropped_img, (resize_width, resize_height))
    result = np.pad(result, pad_spec, mode='constant')
    assert result.shape[0] == height and result.shape[1] == width
    return result

def aumgent_9(impath):
  img = cv.imread(impath,1)
  im1 = cv.flip(img, 0) #Vertical Flip
  im2 = cv.flip(img, 1) #Horizontal Flip
  im3 = cv.flip(img, -1) #Flip Both
  im4 = cv2_clipped_zoom(img, random.uniform(0.9,1.2))
  im5 = cv2_clipped_zoom(im1, random.uniform(0.9,1.2))
  file_name = impath.split('/')
  new_path = file_name[-1].split('-') 
  cv.imwrite("ODIR-5K/Training Images/"+new_path[0]+"VF-"+new_path[1],im1)
  cv.imwrite("ODIR-5K/Training Images/"+new_path[0]+"HF-"+new_path[1],im2)
  cv.imwrite("ODIR-5K/Training Images/"+new_path[0]+"BF-"+new_path[1],im3)
  cv.imwrite("ODIR-5K/Training Images/"+new_path[0]+"ZM-"+new_path[1],im4)
  cv.imwrite("ODIR-5K/Training Images/"+new_path[0]+"VFZM-"+new_path[1],im5)
  #image random rotating and brightening using PIL
  img_org = Image.open(impath)
  enhancer = ImageEnhance.Brightness(img_org)
  im6 = enhancer.enhance(random.uniform(1.0,1.3))
  im7 = img_org.rotate(random.uniform(0,MAX_ROTATE))
  img_ver = Image.open("ODIR-5K/Training Images/"+new_path[0]+"VF-"+new_path[1])
  im8 = img_ver.rotate(random.uniform(0,MAX_ROTATE))
  im6.save("ODIR-5K/Training Images/"+new_path[0]+"RB-"+new_path[1])
  im7.save("ODIR-5K/Training Images/"+new_path[0]+"RR-"+new_path[1])
  im8.save("ODIR-5K/Training Images/"+new_path[0]+"VFRR-"+new_path[1])

#for H class that has the smallest set
def aumgent_20(impath):
  img = cv.imread(impath,1)
  im1 = cv.flip(img, 0) #Vertical Flip
  im2 = cv.flip(img, 1) #Horizontal Flip
  im3 = cv.flip(img, -1) #Both Flip
  im4 = cv2_clipped_zoom(img, random.uniform(0.9,1.2))
  im5 = cv2_clipped_zoom(im1, random.uniform(0.9,1.2))
  im6 = cv2_clipped_zoom(im2, random.uniform(0.9,1.2))
  im7 = cv2_clipped_zoom(im3, random.uniform(0.9,1.2))
  file_name = impath.split('/')
  new_path = file_name[-1].split('-') 
  cv.imwrite("ODIR-5K/Training Images/"+new_path[0]+"VF-"+new_path[1],im1)
  cv.imwrite("ODIR-5K/Training Images/"+new_path[0]+"HF-"+new_path[1],im2)
  cv.imwrite("ODIR-5K/Training Images/"+new_path[0]+"BF-"+new_path[1],im3)
  cv.imwrite("ODIR-5K/Training Images/"+new_path[0]+"ZM-"+new_path[1],im4)
  cv.imwrite("ODIR-5K/Training Images/"+new_path[0]+"VFZM-"+new_path[1],im5)
  cv.imwrite("ODIR-5K/Training Images/"+new_path[0]+"HFZM-"+new_path[1],im6)
  cv.imwrite("ODIR-5K/Training Images/"+new_path[0]+"BFZM-"+new_path[1],im7)
  #image random rotating and brightening using PIL
  img_org = Image.open(impath)
  enhancer = ImageEnhance.Brightness(img_org)
  im7 = enhancer.enhance(random.uniform(0.9,1.2))
  im8 = img_org.rotate(random.uniform(0,MAX_ROTATE))
  img_ver = Image.open("ODIR-5K/Training Images/"+new_path[0]+"VF-"+new_path[1])
  im9 = img_ver.rotate(random.uniform(0,MAX_ROTATE))
  im10 = ImageEnhance.Brightness(im9).enhance(random.uniform(0.9,1.2))
  im11 =  ImageEnhance.Brightness(img_ver).enhance(random.uniform(0.9,1.2))
  img_hor = Image.open("ODIR-5K/Training Images/"+new_path[0]+"HF-"+new_path[1])
  im12 = img_hor.rotate(random.uniform(0,MAX_ROTATE))
  im13 = ImageEnhance.Brightness(im12).enhance(random.uniform(0.9,1.2))
  im14 = ImageEnhance.Brightness(img_hor).enhance(random.uniform(0.9,1.2))
  im_bfl =  Image.open("ODIR-5K/Training Images/"+new_path[0]+"BF-"+new_path[1])
  im15 = im_bfl.rotate(random.uniform(0,MAX_ROTATE))
  im16 = ImageEnhance.Brightness(im15).enhance(random.uniform(0.9,1.2))
  im17 = ImageEnhance.Brightness(im_bfl).enhance(random.uniform(0.9,1.2))
  im_zom =  Image.open("ODIR-5K/Training Images/"+new_path[0]+"ZM-"+new_path[1])
  im18 = im_zom.rotate(random.uniform(0,MAX_ROTATE))
  im19 = ImageEnhance.Brightness(im18).enhance(random.uniform(0.9,1.2))
  im20 = ImageEnhance.Brightness(im_zom).enhance(random.uniform(0.9,1.2))

  im7.save("ODIR-5K/Training Images/"+new_path[0]+"RB-"+new_path[1])
  im8.save("ODIR-5K/Training Images/"+new_path[0]+"RR-"+new_path[1])
  im9.save("ODIR-5K/Training Images/"+new_path[0]+"VFRR-"+new_path[1])
  im10.save("ODIR-5K/Training Images/"+new_path[0]+"VFRRRB-"+new_path[1])
  im11.save("ODIR-5K/Training Images/"+new_path[0]+"VFRB-"+new_path[1])
  im12.save("ODIR-5K/Training Images/"+new_path[0]+"HFRR-"+new_path[1])
  im13.save("ODIR-5K/Training Images/"+new_path[0]+"HFRRRB-"+new_path[1])
  im14.save("ODIR-5K/Training Images/"+new_path[0]+"HFRB-"+new_path[1])
  im15.save("ODIR-5K/Training Images/"+new_path[0]+"BFRR-"+new_path[1])
  im16.save("ODIR-5K/Training Images/"+new_path[0]+"BFRRRB-"+new_path[1])
  im17.save("ODIR-5K/Training Images/"+new_path[0]+"BFRB-"+new_path[1])
  im18.save("ODIR-5K/Training Images/"+new_path[0]+"ZMRR-"+new_path[1])
  im19.save("ODIR-5K/Training Images/"+new_path[0]+"ZMRRRB-"+new_path[1])
  im20.save("ODIR-5K/Training Images/"+new_path[0]+"ZMRB-"+new_path[1])

#aumgent_1 will be applied to class D and O, since they don't need to be aumgmented so much in order to balance the dataset
def aumgent_1(impath):
  img = cv.imread(impath,0)
  im1 = cv.flip(img, 0) #Vertical Flip
  file_name = impath.split('/')
  new_path = file_name[-1].split('-') 
  cv.imwrite("ODIR-5K/Training Images/"+new_path[0]+"VF-"+new_path[1],im1)


In [ ]:
labels = ["G.jpg", "C.jpg", "A.jpg","M.jpg"]
labels2 = ["D.jpg", "O.jpg"]

# Augment data for diseases with a lower count of images
for element in glob.glob("ODIR-5K/Training Images/*.jpg"):
  file_name = element.split('/')
  new_path = file_name[-1].split('-')
  if new_path[-1] =="H.jpg":
    aumgent_20(element)
  else:
    for l in labels:
      if new_path[-1] == l:
        aumgent_9(element)
    for l in labels2:
      if new_path[-1] == l:
        aumgent_1(element)

In [ ]:
show_class_distribution()

1999 1982 1341 1557 1431 1364 1494 1072 379


In [ ]:
!zip -r ODIR-5K_aug_contrast.zip ODIR-5K/

NOTE: We assume you have a directory called "ML490" in your Google Drive where the augmented data will be stored. If you do not, adjust the path below to point to where you will save it. If you change it, you must make a modification in the model script(s) to account for this path change

In [ ]:
!cp -r ODIR-5K_aug_contrast.zip /amd/My\ Drive/ML490

In [ ]:
!rm -rf ODIR-5K/